# Apache Beam
1. 어디에 사용되는가:
    - 데이터 처리(검증과 전처리) 단계에서 작업 정의
    - 파이프라인 오케스트레이션
    - 사용자 정의 컴포넌트 작성

2. 기본 개념
    - 컬렉션(collection): Beam이 읽거나 쓰기 위해 사용하는 데이터 형식
    - 변환(transform): collection을 소비하여 이뤄지는 데이터 조작

이 코드를 파이썬 스크립트로 모아 `python pipeline.py`으로 실행할 수 있다.
그 경우 파이프라인 실행은 DirectRunner가 담당한다.

In [1]:
import re

import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions

In [2]:
def format_result(word_count):
    word, count = word_count
    return f"{word}: {count}"

In [3]:
input_file = "gs://dataflow-samples/shakespeare/kinglear.txt"
output_file = "output.txt"

### 텍스트 파일 내 단어 등장 빈도를 계산하는 작업을 파이프라인으로 정의

In [4]:
#  PipelineOptions 객체의 `runner` 인자를 수정해 실행기를 Spark, Flink로 변경할 수 있다.
pipeline_options = PipelineOptions()

# 컬렉션을 처리하는 변환은 모두 파이프라인 컨텍스트 하에서 실행된다.
with beam.Pipeline() as p:
    # 입력 컬렉션: 빔이 처리하는 컬렉션 데이터는 `PCollection` 타입으로 나타낸다.
    lines = p | beam.io.ReadFromText(input_file)

    # chaining을 이용한 변환 과정 기술:
    # `out_collection =
    #       (in_collection | 'desc' >> process | 'desc' >> process ...)`
    counts = (
        lines
        | "Split" >> beam.FlatMap(lambda x: re.findall(r"A[-Za-z]+", x))
        | "PairWithOne" >> beam.Map(lambda x: (x, 1))
        | "GroupAndSum" >> beam.CombinePerKey(sum)
    )
    # 파이썬 함수로 원하는 작업을 직접 기술할 수 있다.
    output = counts | 'Format' >> beam.Map(format_result)

    # 출력 컬렉션
    output | WriteToText(output_file)